## An introduction to Turing 
[Turing](https://github.com/yebai/Turing.jl) is a general-purpose probabilistic programming language based on Julia.

Here we describe how to install Julia and Turing and run a very simple probabilistic program. 

### Prepration
Install Julia from http://julialang.org/downloads/ 


In [ ]:
Pkg.update()

Install Turing 

In [ ]:
Pkg.add("Turing")


Install some useful Julia packages:

In [ ]:
Pkg.add("IJulia")
Pkg.add("Gadfly")

#### Create your own model
A probabilistic program is Julia code wrapped in a @model macro. It can use arbitrary Julia code, but to ensure correctness of inference it should not have external effects or modify global state. 

For probabilistic effects, Turing programs should use the following macros:
- `@assume x ~ distr` where x is a symbol and distr is a distribution. Inside the probabilistic program this puts a random variable named x, distributed according to distr, in the current scope. distr can be a value of any type that implements rand(distr), which samples a value from the distribution distr.
- `@observe y ~ distr` This is used for conditioning in a style similar to Anglican. Here y should be a value that is observed to have been drawn from the distribution distr. The likelihood is computed using pdf(distr,y) and should always be positive to ensure correctness of inference algorithms. The observe statements should be arranged so that every possible run traverses all of them in exactly the same order. This is equivalent to demanding that they are not placed inside stochastic control flow.
- `@predict x` Registers the current value of x to be inspected in the results of inference.
Inference methods are functions which take the probabilistic program as one of the arguments.

You can create your own @model using any distribution within the Distributions package. The list of such distributions supported is huge: 
http://distributionsjl.readthedocs.io/en/latest/ 

In [ ]:
using Turing, Distributions

@model gaussdemo begin
  @assume s ~ InverseGamma(2,3)
  @assume m ~ Normal(0,sqrt(s))
  @observe 1.5 ~ Normal(m, sqrt(s))
  @observe 2.0 ~ Normal(m, sqrt(s))
  @predict s m
end

# Sample and print.
res = sample(gaussdemo, SMC(10000))
println("Infered: m = $(mean(res[:m])), s = $(mean(res[:s]))")

# Compute analytical solution. Requires `ConjugatePriors` package.
exact = posterior(NormalInverseGamma(0,1,2,3), Normal, [1.5,2.0])
println("Exact: m = $(mean(exact)[1]), s = $(mean(exact)[2])")


## Inference

In [ ]:
# Run sampler, collect results
chain = sample(gaussdemo, SMC(500))

# Print results.
println("Infered: m = $(mean(res[:m])), s = $(mean(res[:s]))")
